In [19]:
import pandas as pd
import numpy as np
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import zipfile
import cv2
from tqdm import tqdm

import tensorflow as tf
from tensorflow import keras

import keras as k
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
import datetime

from tensorboard.plugins.hparams import api_pb2
from keras.callbacks import TensorBoard
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext tensorboard
from tensorboard.plugins.hparams import summary as hparams_summary
from google.protobuf import struct_pb2
from keras.callbacks import TensorBoard

import tensorflow as tf
import datetime

!rm -rf ./logs/

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
train_df = pd.read_csv("/content/drive/My Drive/data/Planet/train_v2.csv/train_v2.csv")
train_df.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [6]:
test_df = pd.read_csv("/content/drive/My Drive/data/Planet/sample_submission_v2.csv/sample_submission_v2.csv")
test_df.head()

,image_name,tags
0,test_0,primary clear agriculture road water
1,test_1,primary clear agriculture road water
2,test_2,primary clear agriculture road water
3,test_3,primary clear agriculture road water
4,test_4,primary clear agriculture road water


In [0]:
#!unzip -uq "/content/drive/My Drive/data/Planet/train-jpg/train-jpg.zip" -d "/content/drive/My Drive/data/Planet/train-jpg/train-jpg"

In [0]:
# Extracting the files
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
link = 'https://drive.google.com/open?id=1DWmr3LOsL7A3ATpLj1S-Np1J4c-kZQeQ'
fluff, id = link.split('=')

download = drive.CreateFile({'id':id})
download.GetContentFile('train-jpg.zip')

with zipfile.ZipFile('train-jpg.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [0]:
xTrain = []
yTrain = []

labels = train_df['tags'].str.get_dummies(sep=' ').columns

In [9]:
labels_map = {l: i for i, l in enumerate(labels)}
inv_labels_map = {i: l for l, i in labels_map.items()}
inv_labels_map

{0: 'agriculture',
 1: 'artisinal_mine',
 2: 'bare_ground',
 3: 'blooming',
 4: 'blow_down',
 5: 'clear',
 6: 'cloudy',
 7: 'conventional_mine',
 8: 'cultivation',
 9: 'habitation',
 10: 'haze',
 11: 'partly_cloudy',
 12: 'primary',
 13: 'road',
 14: 'selective_logging',
 15: 'slash_burn',
 16: 'water'}

In [10]:

for f, tags in tqdm(train_df.values, miniters=1000):
    img = cv2.imread('train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[labels_map[t]] = 1 
    xTrain.append(cv2.resize(img, (32, 32)))
    yTrain.append(targets)

xTrain = np.array(xTrain, np.float16) / 255.
yTrain = np.array(yTrain, np.uint8)

100%|██████████| 40479/40479 [00:56<00:00, 712.12it/s]


In [0]:
link = 'https://drive.google.com/open?id=1qJraDETkZL4xStB54c7i4WXYch51-_PW'
fluff, id = link.split('=')

download = drive.CreateFile({'id':id}) 
download.GetContentFile('test-jpg.zip')

with zipfile.ZipFile('test-jpg.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [0]:
xTest = []
yTest = []

In [13]:
for f, tags in tqdm(train_df.values, miniters=1000):
  try:
    img = cv2.imread('test-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[labels_map[t]] = 1 
    xTest.append(cv2.resize(img, (32, 32)))
    yTest.append(targets)
  except:
    continue
    
xTest = np.array(xTest, np.float16) / 255.
yTest = np.array(yTest, np.uint8)

100%|██████████| 40479/40479 [00:03<00:00, 11861.88it/s]


In [0]:
#Split the dataset to train and test
x_train, x_valid, y_train, y_valid = xTrain[:35000], xTrain[35000:], yTrain[:35000], yTrain[35000:]

In [0]:
def planet_model():
  my_model = Sequential()
  my_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3)))
  my_model.add(Conv2D(64, (3, 3), activation='relu'))
  my_model.add(MaxPooling2D(pool_size=(2, 2)))
  my_model.add(Dropout(0.25))
  my_model.add(Flatten())
  my_model.add(Dense(128, activation='relu'))
  my_model.add(Dropout(0.5))
  my_model.add(Dense(17, activation='sigmoid'))
  return my_model

In [16]:
model = planet_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 12544)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1605760   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)              

In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
#Train the model
model.fit(x_train, y_train, batch_size=128, epochs=4, verbose=1, validation_data=(x_valid, y_valid))

Train on 35000 samples, validate on 5479 samples
Epoch 1/4
35000/35000 [==============================] - 121s 3ms/step - loss: 0.2541 - accuracy: 0.9035 - val_loss: 0.2026 - val_accuracy: 0.9197
Epoch 2/4
35000/35000 [==============================] - 120s 3ms/step - loss: 0.2039 - accuracy: 0.9200 - val_loss: 0.1829 - val_accuracy: 0.9279
Epoch 3/4
35000/35000 [==============================] - 120s 3ms/step - loss: 0.1902 - accuracy: 0.9256 - val_loss: 0.1738 - val_accuracy: 0.9306
Epoch 4/4
35000/35000 [==============================] - 118s 3ms/step - loss: 0.1836 - accuracy: 0.9281 - val_loss: 0.1669 - val_accuracy: 0.9343


In [0]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=logdir, histogram_freq=1)

In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
model.fit(x_train, y_train, batch_size=128, epochs=4, verbose=1, validation_data=(x_valid, y_valid))

Train on 35000 samples, validate on 5479 samples
Epoch 1/4
35000/35000 [==============================] - 120s 3ms/step - loss: 0.1787 - accuracy: 0.9305 - val_loss: 0.1662 - val_accuracy: 0.9347
Epoch 2/4
35000/35000 [==============================] - 119s 3ms/step - loss: 0.1725 - accuracy: 0.9323 - val_loss: 0.1620 - val_accuracy: 0.9365
Epoch 3/4
35000/35000 [==============================] - 119s 3ms/step - loss: 0.1695 - accuracy: 0.9333 - val_loss: 0.1629 - val_accuracy: 0.9370
Epoch 4/4
35000/35000 [==============================] - 119s 3ms/step - loss: 0.1658 - accuracy: 0.9346 - val_loss: 0.1532 - val_accuracy: 0.9394


In [0]:
#Define model
def planet_model_1():
  my_model_1 = Sequential()
  my_model_1.add(Conv2D(64, (3, 3), input_shape=(32, 32, 3),activation='relu'))
  my_model_1.add(MaxPooling2D(pool_size=(2, 2)))
  my_model_1.add(Conv2D(32, (3, 3), activation='relu'))
  my_model_1.add(MaxPooling2D(pool_size=(2, 2)))
  my_model_1.add(Flatten())
  my_model_1.add(Dense(units=128, activation='relu'))
  my_model_1.add(Dropout(0.5))
  my_model_1.add(Dense(units=128, activation='relu'))
  my_model_1.add(Dropout(0.3))
  my_model_1.add(Dense(units=17, activation='sigmoid'))
  return my_model_1

In [24]:
#Get summary of model
model_1 = planet_model_1()
model_1.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 30, 30, 64)        1792      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 13, 32)        18464     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1152)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               147584    
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)              

In [25]:
#Compile the model
model_1.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
model_1.fit(x_train, y_train,batch_size=128,epochs=5,verbose=1,validation_data=(x_valid, y_valid),callbacks=[tensorboard_callback])


Train on 35000 samples, validate on 5479 samples
Epoch 1/5
35000/35000 [==============================] - 56s 2ms/step - loss: 0.6215 - accuracy: 0.7285 - val_loss: 0.4187 - val_accuracy: 0.9056
Epoch 2/5
35000/35000 [==============================] - 56s 2ms/step - loss: 0.3718 - accuracy: 0.8779 - val_loss: 0.2926 - val_accuracy: 0.9056
Epoch 3/5
35000/35000 [==============================] - 56s 2ms/step - loss: 0.3275 - accuracy: 0.8897 - val_loss: 0.2820 - val_accuracy: 0.9056
Epoch 4/5
35000/35000 [==============================] - 56s 2ms/step - loss: 0.3107 - accuracy: 0.8942 - val_loss: 0.2733 - val_accuracy: 0.9056
Epoch 5/5
35000/35000 [==============================] - 56s 2ms/step - loss: 0.2974 - accuracy: 0.8982 - val_loss: 0.2643 - val_accuracy: 0.9056


In [26]:

model_2 = planet_model_1()
model_2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_2.fit(x_train, y_train,batch_size=128,epochs=10,verbose=1,validation_data=(x_valid, y_valid),callbacks=[tensorboard_callback])

Train on 35000 samples, validate on 5479 samples
Epoch 1/10
35000/35000 [==============================] - 56s 2ms/step - loss: 0.2715 - accuracy: 0.8983 - val_loss: 0.2218 - val_accuracy: 0.9116
Epoch 2/10
35000/35000 [==============================] - 56s 2ms/step - loss: 0.2187 - accuracy: 0.9155 - val_loss: 0.2004 - val_accuracy: 0.9196
Epoch 3/10
35000/35000 [==============================] - 55s 2ms/step - loss: 0.2067 - accuracy: 0.9186 - val_loss: 0.1963 - val_accuracy: 0.9226
Epoch 4/10
35000/35000 [==============================] - 56s 2ms/step - loss: 0.1973 - accuracy: 0.9225 - val_loss: 0.1900 - val_accuracy: 0.9259
Epoch 5/10
35000/35000 [==============================] - 54s 2ms/step - loss: 0.1903 - accuracy: 0.9252 - val_loss: 0.1752 - val_accuracy: 0.9315
Epoch 6/10
35000/35000 [==============================] - 54s 2ms/step - loss: 0.1838 - accuracy: 0.9280 - val_loss: 0.1724 - val_accuracy: 0.9319
Epoch 7/10
35000/35000 [==============================] - 54s 2ms/ste

In [0]:

def amazon_model_2():
  my_model_2 = Sequential()
  my_model_2.add(Conv2D(64, (3, 3), input_shape=(32, 32, 3),activation='relu'))
  my_model_2.add(MaxPooling2D(pool_size=(2, 2)))
  my_model_2.add(Conv2D(32, (3, 3), activation='relu'))
  my_model_2.add(MaxPooling2D(pool_size=(2, 2)))
  my_model_2.add(Flatten())
  my_model_2.add(Dense(units=128, activation='selu'))
  my_model_2.add(Dropout(0.5))
  my_model_2.add(Dense(units=128, activation='relu'))
  my_model_2.add(Dropout(0.3))
  my_model_2.add(Dense(units=17, activation='sigmoid'))
  return my_model_2

In [28]:

model_3 = amazon_model_2()
model_3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_3.fit(x_train, y_train,batch_size=128,epochs=10,verbose=1,validation_data=(x_valid, y_valid),callbacks=[tensorboard_callback])

Train on 35000 samples, validate on 5479 samples
Epoch 1/10
35000/35000 [==============================] - 56s 2ms/step - loss: 0.2523 - accuracy: 0.9041 - val_loss: 0.2031 - val_accuracy: 0.9192
Epoch 2/10
35000/35000 [==============================] - 54s 2ms/step - loss: 0.2029 - accuracy: 0.9199 - val_loss: 0.1838 - val_accuracy: 0.9273
Epoch 3/10
35000/35000 [==============================] - 54s 2ms/step - loss: 0.1884 - accuracy: 0.9256 - val_loss: 0.1757 - val_accuracy: 0.9282
Epoch 4/10
35000/35000 [==============================] - 54s 2ms/step - loss: 0.1804 - accuracy: 0.9288 - val_loss: 0.1685 - val_accuracy: 0.9335
Epoch 5/10
35000/35000 [==============================] - 54s 2ms/step - loss: 0.1741 - accuracy: 0.9314 - val_loss: 0.1738 - val_accuracy: 0.9283
Epoch 6/10
35000/35000 [==============================] - 53s 2ms/step - loss: 0.1714 - accuracy: 0.9324 - val_loss: 0.1628 - val_accuracy: 0.9357
Epoch 7/10
35000/35000 [==============================] - 53s 2ms/ste

In [29]:
!tensorboard dev upload --logdir logs

2020-05-18 17:17:59.849830: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1

***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

logs

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) yes

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=373649185512-8v619h5kft38l4456nm2dj4ubeqsrvh6.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scop

In [34]:
!tensorboard dev update-metadata --name "Keras_amazonplanet_model" --description "Model tuning by using different hyperparameters"

2020-05-18 17:24:56.079135: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Error: Must specify experiment to modify via `--experiment_id`.
